In [1]:
from pathlib import Path
import json

import h5py
import numpy as np
from tqdm import tqdm

from core.acoustics import rt60, edt, c50, d50


np.random.seed(42)

In [2]:
# --------------------------
# File paths
# --------------------------
DATASET_PATH = "data/rir_dataset.h5"
OUTPUT_PATH = "data/rir_metrics.h5"
FS = 16000

# --------------------------
# Load RIRs
# --------------------------
with h5py.File(DATASET_PATH, "r") as f:
    rirs = f["rirs"][:]  # shape: (N, 8192)

n_samples = rirs.shape[0]

# --------------------------
# Allocate metric arrays
# --------------------------
rt60_vals = np.zeros(n_samples, dtype=np.float32)
edt_vals = np.zeros(n_samples, dtype=np.float32)
c50_vals = np.zeros(n_samples, dtype=np.float32)
d50_vals = np.zeros(n_samples, dtype=np.float32)

# --------------------------
# Compute metrics
# --------------------------
for i in tqdm(range(n_samples), desc="Computing acoustic metrics"):
    signal = rirs[i]
    rt60_vals[i] = rt60(signal, fs=FS)
    edt_vals[i] = edt(signal, fs=FS)
    c50_vals[i] = c50(signal, fs=FS)
    d50_vals[i] = d50(signal, fs=FS)

# --------------------------
# Save metrics to HDF5
# --------------------------
with h5py.File(OUTPUT_PATH, "w") as f_out:
    f_out.create_dataset("rt60", data=rt60_vals)
    f_out.create_dataset("edt", data=edt_vals)
    f_out.create_dataset("c50", data=c50_vals)
    f_out.create_dataset("d50", data=d50_vals)

Computing acoustic metrics: 100%|██████████| 50000/50000 [00:20<00:00, 2496.79it/s]
